# Pytorch とは

オブジェクト指向型の深層学習ライブラリ。
テンソル計算と自動微分をサポートするフレームワークで、PyTorch ではこれを使って機械学習のモデルやアルゴリズムを構築する。


In [24]:
# pytorch
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# Utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
%matplotlib inline

# ネットワークの作成


## nn.Sequential を使った書き方


In [25]:
# ランダムシードの設定
torch.manual_seed(42)

# 簡単なネットワークの設定
model = nn.Sequential(
    nn.Linear(3, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

# ネットワーク構成の確認
model

Sequential(
  (0): Linear(in_features=3, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [26]:
# ネットワークのパラメータの確認
for idx, param in enumerate(model.named_parameters()):  # named_parameters()メソッドはmodelのパラメータを返すgenerator
  print(f'param_{idx}:, {param}')

param_0:, ('0.weight', Parameter containing:
tensor([[ 0.4414,  0.4792, -0.1353],
        [ 0.5304, -0.1265,  0.1165],
        [-0.2811,  0.3391,  0.5090],
        [-0.4236,  0.5018,  0.1081],
        [ 0.4266,  0.0782,  0.2784],
        [-0.0815,  0.4451,  0.0853],
        [-0.2695,  0.1472, -0.2660],
        [-0.0677, -0.2345,  0.3830]], requires_grad=True))
param_1:, ('0.bias', Parameter containing:
tensor([-0.4557, -0.2662, -0.1630, -0.3471,  0.0545, -0.5702,  0.5214, -0.4904],
       requires_grad=True))
param_2:, ('2.weight', Parameter containing:
tensor([[ 0.2730,  0.0588, -0.1148,  0.2185,  0.0551,  0.2857,  0.0387, -0.1115]],
       requires_grad=True))
param_3:, ('2.bias', Parameter containing:
tensor([0.0950], requires_grad=True))


## カスタムレイヤーについて


nn.Module を継承することで OOP に基づいたネットワーク層の記述ができる。


In [47]:
class SimpleRegression(nn.Module):
  """
  Simple regression model consists of two layers with ReLU.
  The input size is 3 and the output size is 1.
  """
  def __init__(self):
    super().__init__()
    # 一層目の定義
    self.layer1 = nn.Sequential(
      nn.Linear(3, 4),
      nn.ReLU()
    )
    # 二層目の定義
    self.layer2 = nn.Sequential(
      nn.Linear(4, 1),
      nn.ReLU()
    )

  def forward(self, x):
    x = self.layer1(x)
    out = self.layer2(x)
    return out

In [48]:
torch.manual_seed(123)
input = torch.randn(1, 3)  # tensor([[-0.1115,  0.1204, -0.3696]])

model = SimpleRegression()
output = model(input)
print(output)  # tensor([[0.3778]], grad_fn=<ReluBackward0>)

tensor([[0.3778]], grad_fn=<ReluBackward0>)


## なぜ forward メソッドで予測ができるのか


Python では、クラスに特殊メソッド**call**を定義する事でクラスを関数のように使う事ができる。
上記のカスタムクラスで継承している親クラス torch.nn.Module ではこの**call**メソッドを定義しており、その中で forward メソッドが呼ばれる事で処理が走っている。


## データの扱いについて

データの前処理とバッチ処理は、PyTorch でニューラルネットワークモデルを訓練する際の重要なステップである。

このプロセスを効率化し、柔軟性を提供するために、PyTorch は torch.utils.data.Dataset と torch.utils.data.DataLoader の 2 つの強力なクラスを提供している。これらを使用することで、大規模なデータセットを扱いやすくし、訓練プロセスを効率的に行うことができる。


# 誤差逆伝播


## 誤差逆伝播とは

ニューラルネットワークの学習とは、お手本となる教師データを使ってネットワークのパラメータを調整することである。その過程で出力と教師データの誤差を算出し、その勾配を使ってパラメータを更新する。

その際、出力から入力の方向へ更新が進められるためこの方法を誤差逆伝播と呼ぶ。


In [53]:
import pandas as pd
from sklearn.datasets import load_iris

# ボストンデータセットの読み込み
iris = load_iris()
print(iris)

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

### Dataset クラス

特徴量行列（ラベル以外のデータ）X とラベル y を TensorDataset というクラスに渡して、特徴量行列とラベルを一つのデータベース的なものにまとめる事ができる。
PyTorch では、この形式で Data を扱うのが基本となる。


#### Pandas から TensorDataset へ変換


In [7]:
# Load Iris
import seaborn as sns
iris = sns.load_dataset('iris')
print(type(iris))  # Pandasで読込まれている事の確認
iris.head()

<class 'pandas.core.frame.DataFrame'>


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [8]:
# カテゴリ変数を変換
iris.loc[:, 'species'] = iris.loc[:, 'species'].map({'setosa':0, 'versicolor':1, 'virginica':2})
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


##### Pandas -> Numpy -> Tensor

Pandas データセットは、直接データセットに変更できないので、Pandas を Numpy 配列に変換してから Dataset 型に変換を行う


In [9]:
# Numpy変換の為にvaluesというメソッドを使用

X = torch.FloatTensor(iris.drop("species", axis=1).values)
y = torch.LongTensor(iris["species"].values.astype(np.int64))  # dtypeをint64へ変換してから処理

In [10]:
# Datasetを作成
Dataset = torch.utils.data.TensorDataset(X, y)

In [11]:
# 結果を確認
X_sample, y_sample = Dataset[100]
print(X_sample, y_sample)

tensor([6.3000, 3.3000, 6.0000, 2.5000]) tensor(2)


ここまでが、基本的な Dataset の作成方法となる。
一方でこのような Dataset の扱い方は現場ではほとんどの場合なく、もっと便利な自作（カスタム）Dataset というものを作成する。


#### カスタム Dataset について


#


##### Dataset の設定


自作 Dataset を使用することで、前処理（Transformer）等を細かくカスタマイズすることができる。
作成の際は以下の 3 点を設定する。

1. torch.utils.data.Dataset クラスを継承
2. 特殊メソッド**len**()の設定
3. 特殊メソッド**getitem**()の設定

以下、実際にカスタム Dataset の定義を行う。


[参考](../OOP.ipynb)


In [15]:
class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame, features: list[str], labels: list[str]):
        self.features = df[features].values
        self.labels = df[labels].values.astype(np.int64)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        feature = torch.FloatTensor(self.features[idx])
        label = torch.LongTensor(self.labels[idx])
        return feature, label

In [16]:
# インスタンス化
iris_dataset = IrisDataset(iris, ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], ['species'])

In [17]:
# __len__メソッドの確認
len(iris_dataset)

150

In [19]:
# __getitem__メソッドの確認
iris_dataset[100]

(tensor([6.3000, 3.3000, 6.0000, 2.5000]), tensor([2]))

##### 前処理の追加


任意の処理を施す為の
